In [2]:
# imports
# Required libraries to be installed: Pillow, OpenCV
import cv2 as cv
import numpy as np
import glob
import os
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display
import PIL.Image
import io
import math
import imutils
from skimage import measure
from imutils import contours
from numpy.random import uniform
import pdb

In [3]:
# helper functions
def show_image(a, fmt='jpeg'):
    if(len(a.shape)) == 3:
        a = cv.cvtColor(a, cv.COLOR_BGR2RGB)
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
def read_image(index, source, resize):
    image = cv.imread(source[index])
    return cv.resize(image, (0, 0), fx=resize, fy=resize)

In [4]:
# I tried with multiple images in the beginning
task1 = glob.glob(os.path.join('./', "test_data/Task1/*.jpg")) 
task2 = glob.glob(os.path.join('./', "test_data/Task2/*.mp4")) 
task3 = glob.glob(os.path.join('./', "test_data/Task3/*.mp4")) 
task4 = glob.glob(os.path.join('./', "test_data/Task4/*.mp4")) 

task2_modified = glob.glob(os.path.join('./', "Modified/Task2/*.mp4")) 
task2_gt = glob.glob(os.path.join('./', "training_data/Task2/ground-truth/*.txt")) 
hist = glob.glob(os.path.join('./', "Hist/*.jpg")) 
 
templates = glob.glob(os.path.join('./', "Templates/*.jpg")) 
# print(task1_images)

In [5]:
def find_table(frame, show_results=True):
    low_green = (55, 145, 50)
    high_green = (75, 255, 255)
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    # get the mask for green
    mask_table_hsv = cv.inRange(frame_hsv, low_green, high_green)

    kernel = np.ones((15, 15), np.uint8)
    mask_table_hsv = cv.dilate(mask_table_hsv, kernel, iterations=2)
    mask_table_hsv = cv.erode(mask_table_hsv, kernel, iterations=2)
       
    nb_components, output, stats, centroids = cv.connectedComponentsWithStats(mask_table_hsv, connectivity=4)    
    sizes = stats[:, -1]

    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]

    connected_component = np.zeros(output.shape)
    connected_component[output == max_label] = 255
    #     show_image(connected_component)
    connected_component = connected_component.astype(np.int8)
    table = cv.bitwise_and(frame, frame, mask=connected_component)  

    table = cv.bitwise_and(frame, frame, mask=mask_table_hsv)  
    contours, hierarchy = cv.findContours(mask_table_hsv, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0        
    img_contour = np.zeros(table.shape, np.uint8)

    # take the largest in area contour
    if len(contours) > 0:
        cnt = contours[0]
        for i, cont in enumerate(contours):   
            contour_area = cv.contourArea(cont)            
            if contour_area > max_area:
                max_area = contour_area
                cnt = contours[i]     

        epsilon = 0.005 * cv.arcLength(cnt, True) # contour perimeter
        approx = cv.approxPolyDP(cnt, epsilon, True) # get the approximation
#         print(approx)
        hull = cv.convexHull(cnt)
    
        cv.drawContours(table, [approx], 0, (255, 255, 255), -1)
        table_mask = cv.inRange(table, (255, 255, 255), (255, 255, 255))
#         show_image(table_mask)
        table2 = cv.bitwise_and(frame, frame, mask=table_mask)  
    
        topmost = tuple(cnt[cnt[:,:,1].argmin()][0])
        
        x1 = 9999
        y1 = 100
        
        x2 = 0
        y2 = 0
        
        x3 = 99999
        y3 = 0
        
        x4 = 0
        y4 = 0
        
        for i in cnt:
            # 1 - topleft
            if (i[0][1] < (topmost[1] + 10)):
                if (i[0][0] < x1):
                    x1 = i[0][0]
                    y1 = i[0][1]
            
            # 2 - top right
            if (i[0][1] < (topmost[1] + 10)):
                if (i[0][0] > x2):
                    x2 = i[0][0]
                    y2 = i[0][1]
            
            # 3 - bottom left
            if (i[0][0] < x3):
                x3 = i[0][0]
                y3 = i[0][1]
            
            # 4 - bottom right
            if (i[0][0] > x4):
                x4 = i[0][0]
                y4 = i[0][1]
            ### another way to take p3 and p4 that works even with people. Must make sure table is extracted correctly
#             # 3 - bottom left
#             if (i[0][0] < x3):
#                 x3 = i[0][0]
                
#             if (i[0][1] > y3):
#                 y3 = i[0][1]
            
#             # 4 - bottom right
#             if (i[0][0] > x4):
#                 x4 = i[0][0]
#             if (i[0][1] > y4):
#                 y4 = i[0][1]
            
        #middle 
        x_mid, y_mid = line_intersection(((x1, y1), (x4, y4)), ((x2, y2), (x3, y3)))
        # 5 - middle left
        x5, y5 = line_intersection(((0, y_mid), (table.shape[1], y_mid)), ((x1, y1), (x3, y3)))
        # 6 - middle right
        x6, y6 = line_intersection(((0, y_mid), (table.shape[1], y_mid)), ((x2, y2), (x4, y4)))
                
        pockets = [(x1, y1), (x2, y2), (x3, y3), (x4, y4), (x5, y5), (x6, y6)]
        
#         cv.circle(table2,(x1, y1),10,(255,255,255),2)
#         cv.circle(table2,(x2, y2),10,(0,255,255),2)
#         cv.circle(table2,(x3, y3),10,(0,255,0),2)
#         cv.circle(table2,(x4, y4),10,(255,255,255),2)
#         cv.circle(table2,(x5, y5),10,(255,0,255),2)
#         cv.circle(table2,(x6, y6),10,(255,0,255),2)
        
    return table2, pockets, table_mask

In [6]:
def draw_pockets(table, pockets):
    table = table[0].copy()
    cv.circle(table,(pockets[0][0], pockets[0][1]),10,(255,255,255),2)
    cv.circle(table,(pockets[1][0], pockets[1][1]),10,(0,255,255),2)
    cv.circle(table,(pockets[2][0], pockets[2][1]),10,(0,255,0),2)
    cv.circle(table,(pockets[3][0], pockets[3][1]),10,(255,255,255),2)
    cv.circle(table,(pockets[4][0], pockets[4][1]),10,(255,0,255),2)
    cv.circle(table,(pockets[5][0], pockets[5][1]),10,(255,0,255),2)
    return table

In [7]:
### TAKEN FROM STACKOVERFLOW: 
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x.astype(np.int16), y.astype(np.int16)


In [8]:
# This function takes the video path and returns the a list of frames.
def read_frames(video_path, only_table=True):
    frames = []
    cap = cv.VideoCapture(video_path)  
    if cap.isOpened() == False: 
        print("Error opening video stream or file") 
        return frames
    
    while cap.isOpened():  
        ret, frame = cap.read() # Read the frame
        if ret is True:
            if (only_table):
                frames.append(find_table(frame)[0])
            else:
                frames.append(frame)
        else:
            break
    cap.release()
    return frames

def open_video(path, show_table=False):
    cap = cv.VideoCapture(path)
    
    if cap.isOpened() == False: 
        print("Error opening video stream or file") 
    # Display the video
    current_frame = 0
    max_number_of_frame_to_run = 750

    while cap.isOpened(): 

        ret, frame = cap.read() # Read the frame
        
        if ret is True:
            current_frame = current_frame + 1 
            if(show_table == True):
                cv.imshow("Frame", find_table(frame)[0])
            else:
                cv.imshow("Frame", frame)

            if current_frame > max_number_of_frame_to_run:
                break

            if cv.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    # after playing the video, release the video capture    
    cap.release()
    # close all the frames
    cv.destroyAllWindows()

def write_video(path):
    # We are going to write the same video 
    frames = read_frames(path)
    # here we have the extensions and the fourcc for each of it
    video_extension_and_fourcc_dict = {'avi': cv.VideoWriter_fourcc('M', 'J', 'P', 'G'),
                                       'mp4': 0x7634706d}   

    # We need to create a VideoWriter object. 
    # First, we should specify the output file name with its format (eg: 1_fps_1.mp4). 
    # we should specify the FourCC code and the number of frames per second (FPS). 
    # Lastly, the frame size should be passed (width, height).

    video_output_name = "Modified/Task2/" + path.split("/")[-1].split('.')[-2] + "_modified.mp4"
    output_video = cv.VideoWriter(video_output_name, video_extension_and_fourcc_dict["mp4"], 30,
                                  (frames[0].shape[1], frames[0].shape[0]))

    num_frames = len(frames)
    # We know that the first video has 30 fps.
    for i in range(0, num_frames):
        output_video.write(frames[i]) # writing the frame

    # don't forget to release the video writer
    output_video.release()

In [9]:
def get_bounded_circle(circle):
    sq = int(round(math.sqrt((circle[2] * circle[2]) + (circle[2] * circle[2])))) + 15
    x1 = int(circle[0] - (sq / 2))
    y1 = int(circle[1] - (sq / 2))
    
    x2 = int(circle[0] + (sq / 2))
    y2 = int(circle[1] + (sq / 2))
    
    return ((x1, y1), (x2, y2))

def detect_circles(image):
    img = image.copy()
    image = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    image = cv.medianBlur(image, 5)
    circles = cv.HoughCircles(image,cv.HOUGH_GRADIENT,1,10,
                            param1=50,param2=12,minRadius=5,maxRadius=15)
    circle_bounds = []
    if circles is not None:
        circles = np.uint16(np.around(circles))
        
        for i in circles[0, :]:
            cv.circle(img,(i[0],i[1]),i[2],(0,255,0),2)
            cv.circle(img,(i[0],i[1]),2,(0,0,255),3)
            
            crop_img = get_bounded_circle(i)
            circle_bounds.append(crop_img)
            
            cv.rectangle(img, crop_img[0], crop_img[1], (255, 255, 255), 1)
            cv.rectangle(img, crop_img[0], crop_img[1], (255, 255, 255), 1)
            img[crop_img[0][0]:crop_img[1][0], crop_img[0][1]:crop_img[1][1]]
#     show_image(img)
    return circle_bounds

def draw_circles(image, circles):
    img = image.copy()
    if circles is not None:        
        for i in circles:
            cv.circle(img,(i[0],i[1]),i[2],(0,255,0),2)
            cv.circle(img,(i[0],i[1]),2,(0,0,255),3)
            
    return img

def get_circle(img, circles):
    balls = []
    for circle in circles:
        image = img.copy()
        height, width, _ = image.shape
        img1 = image.copy()
        mask = np.zeros((height, width), np.uint8)
        
        cv.rectangle(mask, circle[0], circle[1], (255, 255, 255), thickness = -1)

        masked_data = cv.bitwise_and(img1, img1, mask=mask)

        _, thresh = cv.threshold(mask, 1, 255, cv.THRESH_BINARY)

        contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        x,y,w,h = cv.boundingRect(contours[0])
        crop = masked_data[y:y + h, x: x + w]
        balls.append(crop)
    
    return balls

color_dictionary = {
    "black": 1,
    "white": 1,
    "blue": 0,
    "brown": 0,
    "green": 0,
    "pink": 0,
    "red": 0,
    "yellow": 0
}
template_dictionary = {
    "blue": 0,
    "brown": 0,
    "green": 0,
    "pink": 0,
    "red": 0,
    "yellow": 0
}

def read_templates():
    for i in range(0,len(templates)):
        template_color = templates[i].split('/')[-1].split('.')[0]
        temp = read_image(i, templates, 1)
        template_dictionary[template_color] = temp
    
def is_ball(ball_original):
    for key in template_dictionary:
        temp = template_dictionary[key]
        ball = ball_original.copy()
        ball = cv.resize(ball, (temp.shape[0] + 10, temp.shape[1] + 5), interpolation = cv.INTER_AREA)
        
        temp_gray = cv.cvtColor(temp, cv.COLOR_BGR2GRAY)
        ball_gray = cv.cvtColor(ball, cv.COLOR_BGR2GRAY)
        res = cv.matchTemplate(ball_gray, temp_gray, cv.TM_CCOEFF_NORMED)
        threshold = 0.5
        loc = np.where(res >= threshold)
        if (len(loc[0]) > 0):
#             show_image(ball_original)
            return True
#     show_image(ball_original)
    return False

In [10]:
def get_white_img(frame, ft=False):
    if (ft == True):
        try:
            frame = find_table(frame.copy())[0]
        except:
            frame = find_table2(frame.copy())
        
    low_green = (0, 0, 242)
    high_green = (255, 255, 255)
    
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    mask_table_hsv = cv.inRange(frame_hsv, low_green, high_green)

    kernel = np.ones((15, 15), np.uint8)
    table = cv.bitwise_and(frame, frame, mask=mask_table_hsv)      
    
    cpy = table.copy()     
    frame2 = cv.cvtColor(table, cv.COLOR_BGR2GRAY)
#     show_image(mask_table_hsv)
    cnts = cv.findContours(mask_table_hsv.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    min_distance = 99999
    max_area = 0
    max_cont = -1
    x = 0
    y = 0
    w = 0
    h = 0
    count = 0
    count2 = 0
    returned_cont = []
    if (len(cnts) > 0):
        cnts = contours.sort_contours(cnts)[0]
        for (i, c) in enumerate(cnts):
            (x, y, w, h) = cv.boundingRect(cnts[i])
            x = x - 10
            y = y - 10
            w = w + 20
            h = h + 20
            rec = cv.rectangle(frame, (x, y, w, h), (255, 0, 255), 2)
            intersection = False
            for cont in returned_cont:
                rect = cv.rectangle(frame, (cont[1][0], cont[1][1], cont[1][2], cont[1][3]), (0, 255, 255), 2)
                if(find_point(cont[1][0], cont[1][1], cont[1][2], cont[1][3], x, y)):
                    intersection = True
                if(find_point(cont[1][0], cont[1][1], cont[1][2], cont[1][3], x + w, y)):
                    intersection = True
                if(find_point(cont[1][0], cont[1][1], cont[1][2], cont[1][3], x, y + h)):
                    intersection = True
                if(find_point(cont[1][0], cont[1][1], cont[1][2], cont[1][3], x + w, y + h)):
                    intersection = True
                if(find_point(x, y, w, h, cont[1][0], cont[1][1])):
                    intersection = True
                if(find_point(x, y, w, h, cont[1][0] + cont[1][2], cont[1][1] + cont[1][3])):
                    intersection = True
                if(find_point(x, y, w, h, cont[1][0], cont[1][1] + cont[1][3])):
                    intersection = True
                if(find_point(x, y, w, h, cont[1][2] + cont[1][2], cont[1][1])):
                    intersection = True
               
            if (intersection != True):
                count += 1
                returned_cont.append([rec, (x, y, w, h)])
                
            contour_area = cv.contourArea(c)   
                
    return count

In [11]:
def task1_results2(filename, img):
    count = get_white_img(img)
    written_balls = {
        "white": 0,
        "black": 0,
        "blue": 0,
        "brown": 0,
        "green": 0,
        "pink": 0,
        "red": 0,
        "yellow": 0
    }
    answers_file = open('./andrei_dumitriu_407/Task1/' + filename + '.txt', 'w+')
    answers_file.write(str(count))
    answers_file.write('\n')
    
    if (count >= 1):
        written_balls['white'] = 1
        
    if (count >= 2):
        written_balls['black'] = 1
        
    if (count >= 3):
        written_balls['pink'] = 1
    
    if (count >= 4):
        written_balls['blue'] = 1
    
    if (count >= 5):
        written_balls['brown'] = 1
    
    if (count >= 6):
        written_balls['green'] = 1
    
    if (count >= 7):
        written_balls['yellow'] = 1
    
    if (count >= 8):
        written_balls['red'] = count - 7
    
    answers_file.write(str(written_balls['white']) + " white")
    answers_file.write('\n')
    answers_file.write(str(written_balls['black']) + " black")
    answers_file.write('\n')
    answers_file.write(str(written_balls['pink']) + " pink")
    answers_file.write('\n')
    answers_file.write(str(written_balls['blue']) + " blue")
    answers_file.write('\n')
    answers_file.write(str(written_balls['green']) + " green")
    answers_file.write('\n')
    answers_file.write(str(written_balls['brown']) + " brown")
    answers_file.write('\n')
    answers_file.write(str(written_balls['yellow']) + " yellow")
    answers_file.write('\n')
    answers_file.write(str(written_balls['red']) + " red")
    answers_file.close()
            
def find_point(x1, y1, x2, y2, x, y):
#     print (x1, y1, x2, y2, x, y)
    if (x >= x1 and x <= x1+ x2 and y > y1 and y <= y1 + y2):
        return True
  
    return False

In [12]:
# TASK 1 apply to all files v2
read_templates()
for i in range(0, len(task1)):
    img = read_image(i, task1, 1)
    name = task1[i].split('/')[-1].split('.')[0]
    table = find_table(img)[0]
#     circles = detect_circles(table)
#     balls = get_circle(img, circles)
#     actual_balls = []
#     for ball in balls:
#         if(is_ball(ball) == True):
#             actual_balls.append(ball)
    task1_results2(name, table)


In [13]:
### TASK 2

In [14]:
def track_pockets(video_path):
    subtractor = cv.createBackgroundSubtractorMOG2(history=1, varThreshold=105, detectShadows=False)
    
    bboxes = []
    cap = cv.VideoCapture(video_path)
    ret, first_frame = cap.read() # Read the first frame 
    corners = find_table(first_frame)[1]
    table_mask = find_table(first_frame)[2]
    
    
    x0 = corners[0][0]
    y0 = corners[0][1]
    w0 = 30
    h0 = 25
    
    x1 = corners[1][0] - 50
    y1 = corners[1][1]
    w1 = 30
    h1 = 30
    
    x2 = corners[2][0] 
    y2 = corners[2][1] - 15
    w2 = 50
    h2 = 30
    
    x3 = corners[3][0] - 50
    y3 = corners[3][1] - 15
    w3 = 50
    h3 = 30
    
    
    x4 = corners[4][0] 
    y4 = corners[4][1] - 15
    w4 = 40
    h4 = 30
    
    x5 = corners[5][0] - 50
    y5 = corners[5][1] - 15
    w5 = 50
    h5 = 30
    
    
#     x, y, w, h = cv.selectROI(first_frame) 
    track_window0 = (x0, y0, w0, h0)
    track_window1 = (x1, y1, w1, h1)
    track_window2 = (x2, y2, w2, h2)
    track_window3 = (x3, y3, w3, h3)
    track_window4 = (x4, y4, w4, h4)
    track_window5 = (x5, y5, w5, h5)
    
    roi0 = first_frame[y0: y0 + h0, x0: x0 + w0]
    roi1 = first_frame[y1: y1 + h1, x1: x1 + w1]
    roi2 = first_frame[y2: y2 + h2, x2: x2 + w2]
    roi3 = first_frame[y3: y3 + h3, x3: x3 + w3]
    roi4 = first_frame[y4: y4 + h4, x4: x4 + w4]
    roi5 = first_frame[y5: y5 + h5, x5: x5 + w5]
    
    annotated_frame0 = cv.rectangle(first_frame, (x0, y0), (x0+w0,y0+h0), 255, 2)
    annotated_frame1 = cv.rectangle(first_frame, (x1, y1), (x1+w1,y1+h1), 255, 2)
    annotated_frame2 = cv.rectangle(first_frame, (x2, y2), (x2+w2,y2+h2), 255, 2)
    annotated_frame3 = cv.rectangle(first_frame, (x3, y3), (x3+w3,y3+h3), 255, 2)
    annotated_frame4 = cv.rectangle(first_frame, (x4, y4), (x4+w4,y4+h4), 255, 2)
    annotated_frame5 = cv.rectangle(first_frame, (x5, y5), (x5+w5,y5+h5), 255, 2)
    
    frame_no = 0
    
    
    roi_hist0 = cv.calcHist([roi0], [0 ,1, 2], None, [4, 4, 4], [0, 256, 0, 256, 0, 256]) 
    roi_hist1 = cv.calcHist([roi1], [0 ,1, 2], None, [4, 4, 4], [0, 256, 0, 256, 0, 256]) 
    roi_hist2 = cv.calcHist([roi2], [0 ,1, 2], None, [4, 4, 4], [0, 256, 0, 256, 0, 256]) 
    roi_hist3 = cv.calcHist([roi3], [0 ,1, 2], None, [4, 4, 4], [0, 256, 0, 256, 0, 256]) 
    roi_hist4 = cv.calcHist([roi4], [0 ,1, 2], None, [4, 4, 4], [0, 256, 0, 256, 0, 256]) 
    roi_hist5 = cv.calcHist([roi5], [0 ,1, 2], None, [4, 4, 4], [0, 256, 0, 256, 0, 256]) 
    
    roi_hist_norm0 = roi_hist0 / roi_hist0.sum()
    roi_hist_norm1 = roi_hist1 / roi_hist1.sum()
    roi_hist_norm2 = roi_hist2 / roi_hist2.sum()
    roi_hist_norm3 = roi_hist3 / roi_hist3.sum()
    roi_hist_norm4 = roi_hist4 / roi_hist4.sum()
    roi_hist_norm5 = roi_hist5 / roi_hist5.sum()

    roi_gray0 = cv.cvtColor(roi0, cv.COLOR_BGR2GRAY)
    roi_gray1 = cv.cvtColor(roi1, cv.COLOR_BGR2GRAY)
    roi_gray2 = cv.cvtColor(roi2, cv.COLOR_BGR2GRAY)
    roi_gray3 = cv.cvtColor(roi3, cv.COLOR_BGR2GRAY)
    roi_gray4 = cv.cvtColor(roi4, cv.COLOR_BGR2GRAY)
    roi_gray5 = cv.cvtColor(roi5, cv.COLOR_BGR2GRAY)
    
    
    annotated_frame = first_frame.copy()
    frame_no = 0
#     cv.imshow('First frame initialization', annotated_frame)
#     cv.waitKey(10000)
    
    
    frame_idx = 0
    total_sum = 0
    p0 = []
    p1 = []
    p2 = []
    p3 = []
    p4 = []
    p5 = []
    while cap.isOpened():
        frame_idx += 1
        ret, frame = cap.read()

        if ret is True: 
#             frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            frame = cv.bitwise_and(frame, frame, mask=table_mask)  
            frame_gray = subtractor.apply(frame)
            mask10 = np.int8(np.zeros(frame_gray.shape))
            mask11 = np.int8(np.zeros(frame_gray.shape))
            mask12 = np.int8(np.zeros(frame_gray.shape))
            mask13 = np.int8(np.zeros(frame_gray.shape))
            mask14 = np.int8(np.zeros(frame_gray.shape))
            mask15 = np.int8(np.zeros(frame_gray.shape))
            
            center0 = (y0 + h0//2, x0 + h0//2)
            center1 = (y1 + h1//2, x1 + h1//2)
            center2 = (y2 + h2//2, x2 + h2//2)
            center3 = (y3 + h3//2, x3 + h3//2)
            center4 = (y4 + h4//2, x4 + h4//2)
            center5 = (y5 + h5//2, x5 + h5//2)

            y_min0 = y0 #np.max((0, center0[0] - (2*h0)))
            y_max0 = y0 + h0 #np.min((frame.shape[0], center0[0] + (2*h0)))
            x_min0 = x0 #np.max((0, center0[1] - (2*w0)))
            x_max0 = x0 + w0 #np.min((frame.shape[1], center0[1] + (2*w0)))
                        
            y_min1 = y1 #np.max((0, center1[0] - (2*h1)))
            y_max1 = y1 + h1  #np.min((frame.shape[0], center1[0] + (2*h1)))
            x_min1 = x1 #np.max((0, center1[1] - (2*w1)))
            x_max1 = x1 + w1 #np.min((frame.shape[1], center1[1] + (2*w1)))
                        
            y_min2 = y2 #np.max((0, center2[0] - (2*h2)))
            y_max2 = y2 + h2 #np.min((frame.shape[0], center2[0] + (2*h2)))
            x_min2 = x2 #np.max((0, center2[1] - (2*w2)))
            x_max2 = x2 + w2# np.min((frame.shape[1], center2[1] + (2*w2)))
                        
            y_min3 = y3 #np.max((0, center3[0] - (2*h3)))
            y_max3 = y3 + h3 #np.min((frame.shape[0], center3[0] + (2*h3)))
            x_min3 = x3 #np.max((0, center3[1] - (2*w3)))
            x_max3 = x3 + w3 #np.min((frame.shape[1], center3[1] + (2*w3)))
                        
            y_min4 = y4 #np.max((0, center4[0] - (2*h4)))
            y_max4 = y4 + h4 #np.min((frame.shape[0], center4[0] + (2*h4)))
            x_min4 = x4 # np.max((0, center4[1] - (2*w4)))
            x_max4 = x4 + w4 #np.min((frame.shape[1], center4[1] + (2*w4)))
            
            y_min5 = y5 #np.max((0, center5[0] - (2*h5)))
            y_max5 = y5 + h5 #np.min((frame.shape[0], center5[0] + (2*h5)))
            x_min5 = x5 #np.max((0, center5[1] - (2*w5)))
            x_max5 = x5 + w5 #np.min((frame.shape[1], center5[1] + (2*w5)))
            
            mask10[y_min0: y_max0, x_min0: x_max0] = 255
            mask11[y_min1: y_max1, x_min1: x_max1] = 255
            mask12[y_min2: y_max2, x_min2: x_max2] = 255
            mask13[y_min3: y_max3, x_min3: x_max3] = 255
            mask14[y_min4: y_max4, x_min4: x_max4] = 255
            mask15[y_min5: y_max5, x_min5: x_max5] = 255
            
#             mask1[:, :] = 255
            frame_gray0 = cv.bitwise_and(frame_gray,frame_gray,mask=mask10)
            frame_gray1 = cv.bitwise_and(frame_gray,frame_gray,mask=mask11)
            frame_gray2 = cv.bitwise_and(frame_gray,frame_gray,mask=mask12)
            frame_gray3 = cv.bitwise_and(frame_gray,frame_gray,mask=mask13)
            frame_gray4 = cv.bitwise_and(frame_gray,frame_gray,mask=mask14)
            frame_gray5 = cv.bitwise_and(frame_gray,frame_gray,mask=mask15)
#             cv.imshow('frame gray mask 0', frame_gray0)
#             cv.imshow('frame gray mask 1', frame_gray1)
#             cv.imshow('frame gray mask 2', frame_gray2)
#             cv.imshow('frame gray mask 3', frame_gray3)
#             cv.imshow('frame gray mask 4', frame_gray4)
#             cv.imshow('frame gray mask 5', frame_gray5)
            p0.append(frame_gray0.sum())
            p1.append(frame_gray1.sum())
            p2.append(frame_gray2.sum())
            p3.append(frame_gray3.sum())
            p4.append(frame_gray4.sum())
            p5.append(frame_gray5.sum())
            
            cv.rectangle(frame, (x_min0, y_min0), (x_max0, y_max0), 255, 2)
            cv.rectangle(frame, (x_min1, y_min1), (x_max1, y_max1), 255, 2)
            cv.rectangle(frame, (x_min2, y_min2), (x_max2, y_max2), 255, 2)
            cv.rectangle(frame, (x_min3, y_min3), (x_max3, y_max3), 255, 2)
            cv.rectangle(frame, (x_min4, y_min4), (x_max4, y_max4), 255, 2)
            cv.rectangle(frame, (x_min5, y_min5), (x_max5, y_max5), 255, 2)
            
#             cv.imshow('img', frame)
#             cv.waitKey(500)

#             if cv.waitKey(1) & 0xFF == ord('q'):
#                 break
                
        else:
            break
    # after playing the video, release the video capture    
    cap.release()
    # close all the frames
    cv.destroyAllWindows()
    return ([sum(p0), sum(p1[1:-2]), sum(p2[1:-2]), sum(p3[1:-2]), sum(p4[1:-2]), sum(p5[1:-2])])
#     return bboxes

In [44]:
def task2_video(file):
    pc_sum = track_pockets(file)
    threshold = 300000
    filename = file.split("/")[-1].split('.')[-2]
    max_el = max(pc_sum)
    index = pc_sum.index(max_el)
    print(index)
    if (max_el > threshold):
        return index + 1
    else:
        return -1

In [45]:
task2.sort()

for task in task2[-1:]:
    filename = task.split("/")[-1].split('.')[-2]
    answers_file = open('./andrei_dumitriu_407/Task2/' + filename + '.txt', 'w+')
    print(filename)
    index = task2_video(task)
    if (index > 0):
        answers_file.write('YES')
        answers_file.write('\n')
        answers_file.write(str(index))
        answers_file.write('\n')
        answers_file.write('red')
    else:
        answers_file.write('NO')

9
2


In [22]:
# PARTICLE FILTER FROM THE LAB
# a particle is a bopunding box, represented by the top left corner 
# and fixed width and hieght
def create_uniform_particles(x_range, y_range, N):
    particles = np.empty((N, 2))
    particles[:, 0] = uniform(x_range[0], x_range[1], size=N)
    particles[:, 1] = uniform(y_range[0], y_range[1], size=N) 
    return particles

# predict where the particles will be at the nex frame by applying some dynamics
# take into account velocity and some random noise

def predict(particles, velocity, std, frame, w, h):  
    N = len(particles)    
    
    noise = np.random.randn(N) * std[0]  
    for i in range(N):
        particles[i, 0] = particles[i, 0] + velocity[0] + noise[i]
        # check that the particle is not outside of the image
        if(particles[i, 0] > frame.shape[1] -w):
            particles[i, 0] = frame.shape[1] - w
        if(particles[i, 0] < 0):
            particles[i, 0] = 0
            
    noise = np.random.randn(N) * std[1]
    for i in range(N):
        particles[i, 1] = particles[i, 1] + velocity[1] + noise[i]
        # check the particle is still in the image
        if(particles[i, 1] > frame.shape[0] - h):
            particles[i, 1] = frame.shape[0] - h
        if(particles[i, 1] < 0):
            particles[i, 1] = 0 
    return particles

# update the weight of each particle based on how similar is to the target window
# use a simple color histogram model
# essential step: how to update the weights
def update(particles, frame, hist_roi_norm, w, h):
    particles = np.int32(particles)   
    weights = np.zeros((particles.shape[0],1))
    for i in range(particles.shape[0]):
        img_particle = frame[particles[i, 1]: particles[i, 1] + h - 1, particles[i, 0]:particles[i, 0] + w - 1].copy()
        particle_hist = cv.calcHist([img_particle], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]) 
        particle_hist_norm = particle_hist/ particle_hist.sum()
        distance = cv.compareHist(hist_roi_norm, particle_hist_norm, cv.HISTCMP_CHISQR_ALT)
        # higher chi-square distance is bad, smaller chi-square distance is better
        weights[i] = 1 / (np.power(distance,5))
    weights += 1.e-10 # avoid round-off to zero
    # normalize the wights such that we have a probability distribution
    weights /= sum(weights)
    return weights

# estimate the center of the cloud of particles
def estimate(particles, weights):    
    mean = np.float64(np.array([0,0]))
    N = particles.shape[0]  
    for i in range(N): 
        mean += weights[i] * particles[i, :]   
    return mean

# resample particles based on their weight
def resample(weights):
    w = weights.flatten()
    N = len(w)    
    tries = np.random.multinomial(N, w) 
    indexes = np.zeros(N, 'i')
    cumsum_vector = np.cumsum(tries)
    pos = -1 
    for i in range(len(tries)):
        for j in range(tries[i]):            
            pos = pos + 1
            indexes[pos] = i
            
    return indexes

def resample_from_index(particles, weights, indexes):
    particles[:] = particles[indexes]
    weights[:] = weights[indexes]
    weights /= np.sum(weights)
    return particles, weights

def particle_filter(video, inital_coords):
    # change the path to match on your machine
    cap = cv.VideoCapture(video)  
    current_frame = 0
    max_number_of_frame_to_run = 750

    ret, frame = cap.read() # Read the frame
    frame_gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    old_frame_gray = frame_gray

    ret, first_frame = cap.read() # Read the first frame         
    x, y, x2, y2 = inital_coords
    w = x2 - x
    h = y2 - y
    img_roi = frame[y: y + h, x: x + w].copy()
    hist_roi = cv.calcHist([img_roi], [0 ,1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]) 
    hist_roi_norm = hist_roi/ hist_roi.sum()
    bb = np.array([x, y, x + w - 1, y + h - 1])

    N = 1000
    particles = create_uniform_particles([x, x], [y, y], N)
    velocity = [0, 0]
    std = [10, 10] 
    bboxes = [[x, y, x2, y2]]
    while(cap.isOpened()): 
        ret, frame = cap.read() # Read the frame   
        if ret is True: 
            current_frame = current_frame + 1 
            frame_gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)


            particles = predict(particles, velocity, std, frame, w, h)        

            weights = update(particles, frame, hist_roi_norm, w, h) 

            obj = np.int32(estimate(particles, weights))       

            velocity[0] = obj[0] - bb[0]
            velocity[1] = obj[1] - bb[1]  
            bb = obj.copy()

            indexes = resample(weights)
            particles, weights = resample_from_index(particles, weights, indexes)               

            for i in range(N):            
                img2 = cv.rectangle(frame, (np.int32(particles[i,0]), np.int32(particles[i,1])), (np.int32(particles[i,0]) + w - 1, np.int32(particles[i,1]) + h - 1), (0,255,0), 1)
            bboxes.append([obj[0], obj[1], obj[0] + w - 1, obj[1] + h - 1])
            img2 = cv.rectangle(frame, (obj[0], obj[1]), (obj[0] + w - 1, obj[1] + h - 1), (0, 255, 255), 4)
            cv.imshow('img2', img2)   
            cv.waitKey(1)
            if current_frame > max_number_of_frame_to_run:
                break

#             if cv.waitKey(1) & 0xFF == ord('q'):
#                 break
        else:
            break

    # after playing the video, release the video capture    
    cap.release()                
    # close all the frames
    cv.destroyAllWindows()
    return bboxes

In [23]:
# TASK 3 OUTPUT file 1
task3.sort()
#  v1
for task in task3:
    filename = task.split("/")[-1].split('.')[-2]
    print(filename)
    initial_file = open('./test_data/Task3/' + filename + '_ball_1.txt', 'r')
    initial_file.readline()
    line = initial_file.readline()
    line = line.split(" ")
    initial_point = (int(line[1]), int(line[2]), int(line[3]), int(line[4]))
    pos_array = particle_filter(task, initial_point)
    
    
    answers_file = open('./andrei_dumitriu_407/Task3/' + filename + '_ball_1.txt', 'w+')
    frames = read_frames(task, False)
    answers_file.write(str((len(frames))) + " -1 -1 -1 -1") 
    answers_file.write('\n')
    prev_pos = (0, 0, 0, 0)        
    for i, item in enumerate(pos_array):
        if (item[0] <= 0 or item[1] <= 0 or item[2] <= 0 or item[3] <= 0):
            j = i + 1
            found = False
            indent = 1
            while (found == False and j < len(pos_array) and j > -1):
                if (found == False and pos_array[j][0] > 0 and pos_array[j][1] > 0 and pos_array[j][2] > 0 and pos_array[j][3] > 0):
                    pos_array[i] = pos_array[j]
                    found = True
#                     print("first J", j)
                if (j == len(pos_array)):
                    j = i - 1
                    indent = -1
                j += indent
                    
    for i, item in enumerate(pos_array):
        written_str = str(i) + " " + str(item[0]) + " " + str(item[1]) + " " + str(item[2]) + " " + str(item[3])
        answers_file.write(written_str)
        answers_file.write('\n')
#         print(written_str)
#     print("--------------------------------------------------")
#     print(pos_array)
    answers_file.close()
    initial_file.close()


1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
3
4
5
6
7
8
9


In [26]:
# TASK 3 OUTPUT 2
task3.sort()
#  v1
for task in task3:
    filename = task.split("/")[-1].split('.')[-2]
    print(filename)
    initial_file = open('./test_data/Task3/' + filename + '_ball_2.txt', 'r')
    initial_file.readline()
    line = initial_file.readline()
    line = line.split(" ")
    initial_point = (int(line[1]), int(line[2]), int(line[3]), int(line[4]))
    pos_array = particle_filter(task, initial_point)
    
    
    answers_file = open('./andrei_dumitriu_407/Task3/' + filename + '_ball_2.txt', 'w+')
    frames = read_frames(task, False)
    answers_file.write(str((len(frames))) + " -1 -1 -1 -1") 
    answers_file.write('\n')
    prev_pos = (0, 0, 0, 0)        
    for i, item in enumerate(pos_array):
        if (item[0] <= 0 or item[1] <= 0 or item[2] <= 0 or item[3] <= 0):
            j = i + 1
            found = False
            indent = 1
            while (found == False and j < len(pos_array) and j > -1):
                if (found == False and pos_array[j][0] > 0 and pos_array[j][1] > 0 and pos_array[j][2] > 0 and pos_array[j][3] > 0):
                    pos_array[i] = pos_array[j]
                    found = True
#                     print("first J", j)
                if (j == len(pos_array)):
                    j = i - 1
                    indent = -1
                j += indent
                    
    for i, item in enumerate(pos_array):
        written_str = str(i) + " " + str(item[0]) + " " + str(item[1]) + " " + str(item[2]) + " " + str(item[3])
        answers_file.write(written_str)
        answers_file.write('\n')
#         print(written_str)
#     print("--------------------------------------------------")
#     print(pos_array)
    answers_file.close()
    initial_file.close()


1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
3
4
5
6
7
8
9


In [ ]:
### TASK 4

In [31]:
### TASK 4
def get_white(frame, ft=False):
    if (ft == True):
        try:
            frame = find_table(frame.copy())[0]
        except:
            frame = find_table2(frame.copy())
        
    # best v
#     low_green = (32, 33, 255)
#     high_green = (50, 118, 255)
    
    # actual    
    low_green = (32, 33, 255)
    high_green = (50, 118, 255)
    
    # test versions
#     low_green = (0, 0, 0)
#     high_green = (57, 93, 255)


#     low_green = (32, 33, 199)
#     high_green = (50, 149, 255)
    
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    # get the mask for green
    mask_table_hsv = cv.inRange(frame_hsv, low_green, high_green)

    kernel = np.ones((15, 15), np.uint8)
    mask_table_hsv = cv.dilate(mask_table_hsv, kernel, iterations=2)
    mask_table_hsv = cv.erode(mask_table_hsv, kernel, iterations=2)
       
    table = cv.bitwise_and(frame, frame, mask=mask_table_hsv)      
    
    cpy = table.copy()     
    frame2 = cv.cvtColor(table, cv.COLOR_BGR2GRAY)
#     show_image(mask_table_hsv)
    cnts = cv.findContours(mask_table_hsv.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    min_distance = 99999
    max_area = 0
    max_cont = -1
    x = 0
    y = 0
    w = 0
    h = 0
    if (len(cnts) > 0):
        cnts = contours.sort_contours(cnts)[0]
        # loop over the contours
        for (i, c) in enumerate(cnts):
            (x, y, w, h) = cv.boundingRect(cnts[i])
            
            contour_area = cv.contourArea(c)   
#             print(contour_area)
            if (contour_area > max_area):
                max_area = contour_area
                max_cont = i
        if (max_cont > -1):
            (x, y, w, h) = cv.boundingRect(cnts[max_cont])
            cv.rectangle(frame, (x - 20, y - 10), (x+w + 20, y+h + 30), (255, 0, 255), 2)
            
        cv.imshow('frame', frame)   
        cv.waitKey(1)
    return frame, cpy, (x - 15, y - 10, x + w + 15, y + h + 30)

In [33]:
# TASK 4 OUTPUT
task4.sort()
#  v1
for task in task4:
#     open_video4(task, False)
    filename = task.split("/")[-1].split('.')[-2]
    answers_file = open('./andrei_dumitriu_407/Task4/' + filename + '.txt', 'w+')
    frames = read_frames(task, False)
    answers_file.write(str((len(frames))) + " -1 -1 -1 -1") 
    answers_file.write('\n')
    print(filename)
    pos_array = []
    prev_pos = (0, 0, 0, 0)
    for i, frame in enumerate(frames):
        pos_array.append(get_white(frame)[2])
        
#     print(pos_array)
    for i, item in enumerate(pos_array):
        if (item[0] <= 0 or item[1] <= 0 or item[2] <= 0 or item[3] <= 0):
            j = i
            found = False
            while (found == False and j < len(pos_array)):
                if (pos_array[j][0] > 0 and pos_array[j][1] > 0 and pos_array[j][2] > 0 and pos_array[j][3] > 0):
                    pos_array[i] = pos_array[j]
                    found = True
#                     print("first J", j)
                j += 1
                    
    for i, item in enumerate(pos_array):
        written_str = str(i) + " " + str(item[0]) + " " + str(item[1]) + " " + str(item[2]) + " " + str(item[3]) 
        answers_file.write(written_str)
        answers_file.write('\n')
#         print(written_str)
#     print("--------------------------------------------------")
#     print(pos_array)
    answers_file.close()

        

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
3
4
5
6
7
8
9
